---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    f = open("university_towns.txt", "r") # read the txt file
    
    content = f.readlines() # save each line from the file
    
    f.close()
    
    state_uni_dictionary = {}
    for element in range(0, len(content)):
        if len(content[element].split('[e')) != 1:
            current_state = content[element].split('[e')[0].strip()
            state_uni_dictionary[current_state] = []
        else:
            state_uni_dictionary[current_state].append(content[element].split('[e')[0].strip())
    for key in state_uni_dictionary.keys():
        for element in range(0, len(state_uni_dictionary[key])):
            state_uni_dictionary[key][element] = state_uni_dictionary[key][element].replace('\n','').split(' (')[0]
    
    return pd.DataFrame([[key, value] for key in state_uni_dictionary.keys() for value in state_uni_dictionary[key]], columns=["State", "RegionName"])

In [4]:
def get_GDP_data():
    '''Returns a dataframe with the GDP data from the first quarter of 2000 onward.
    '''
    GDP_data = pd.read_excel("gdplev.xls", skiprows=5).iloc[:,4:8].rename(columns={'Unnamed: 4': 'Quarter', 'GDP in billions of current dollars.1': 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'}).drop(['Unnamed: 7'], axis=1).iloc[2:,].reset_index(drop=True)
    GDP_data = GDP_data[['Quarter', 'GDP in billions of chained 2009 dollars']]
    GDP_data['GDP_Change'] = GDP_data.iloc[(GDP_data.index), 1:2].values - GDP_data.iloc[(GDP_data.index)-1, 1:2].values 
    
    GDP_data['Change'] = ["Increase" if x > 0 else "Decrease" if x < 0 else "Equal" for x in GDP_data['GDP_Change']] 
    # using list comprehension to create dataframe column in a conditional way
    
    GDP_data = GDP_data.where(GDP_data.Quarter.apply(lambda x: x[:4]).astype('int') >= 2000).dropna().reset_index(drop=True)
    # get GDP data from 2000 onward
    
    #GDP_data["Recession_Flag"] = ["Recession Start" if x == "Decrease" and y == "Decrease" and z == "Increase" else "Recession End" if x == "Increase" and y == "Increase" and z == "Decrease" and w == "Decrease" else "----" for x,y,z,w in zip(GDP_data.Change.shift(), GDP_data.Change, GDP_data.Change.shift(2), GDP_data.Change.shift(3))]
    
    return GDP_data

In [5]:
def get_recession_data():
    '''Returns the recession start and end needed in future functions'''
    
    recession_start = []
    recession_end = []
    recession_flag = 0
    index = 0
    for current, next in zip([element for element in get_GDP_data().Change], [element for element in get_GDP_data().Change][1:]):
        if current == 'Decrease' and next == 'Decrease' and recession_flag == 0:
            recession_start.append(index)
            recession_flag = 1
        if current == 'Increase' and next == 'Increase' and recession_flag == 1:
            recession_end.append(index+1)
            recession_flag = 0
        index += 1
        
    return [recession_start, recession_end]

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    #return get_GDP_data().where(GDP_data['Recession_Flag'] == 'Recession Start').dropna().head(1).Quarter.item()
    
    return get_GDP_data().iloc[get_recession_data()[0][0]:get_recession_data()[0][0]+1].Quarter.item()

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    #return GDP_data.where(GDP_data['Recession_Flag'] == 'Recession End').dropna().head(1).Quarter.item()
    
    return get_GDP_data().iloc[get_recession_data()[1][0]:get_recession_data()[1][0]+1].Quarter.item()

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_data = get_GDP_data()[get_GDP_data()[get_GDP_data()['Quarter'] == get_recession_start()].index.item():get_GDP_data()[get_GDP_data()['Quarter'] == get_recession_end()].index.item()+1]
    
    lowest_point = recession_data.iloc[:,0:2].sort_values(by='GDP in billions of chained 2009 dollars', ascending=True).head(1)
    
    return lowest_point.Quarter.item()

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    houses = pd.read_csv("City_Zhvi_AllHomes.csv")
    column_mask = [x for x in (list(houses.columns)[:6] + [x for x in list(houses.columns)[6:] if int(x[:4]) >= 2000])]
    houses = houses.loc[:,column_mask]
    houses.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1 , inplace=True)
    houses.State = houses.State.map(states)
    houses = houses.set_index(['State', 'RegionName'])
    houses.columns = pd.to_datetime(houses.columns).to_period('Q')
    houses = houses.groupby(houses.columns, axis=1).mean()
    houses.columns = houses.columns.map(str)
    
    return houses

In [80]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    house_data = convert_housing_data_to_quarters()
    
    house_data.columns = house_data.columns.str.replace('Q', 'q')
    
    house_data = house_data.loc[:,house_data.columns[(list(house_data.columns).index(get_recession_start())-1)]:get_recession_bottom()]
    
    house_data['Price_Ratio'] = house_data.iloc[:,0]/house_data.iloc[:,-1]
    
    uni_towns = get_list_of_university_towns()
    uni_towns['Univ_Town'] = True
    uni_towns = uni_towns.set_index(["State", "RegionName"])
    uni_towns = uni_towns.reset_index()
    
    house_data = house_data.reset_index()
    
    merged_data = pd.merge(house_data, uni_towns, how='outer', on=['State','RegionName'])
    merged_data.Univ_Town = merged_data.Univ_Town.replace(np.nan, False)
    
    #inner_data = pd.merge(house_data, uni_towns, on=['State','RegionName'])
    
    t, P = ttest_ind(merged_data[merged_data['Univ_Town']].Price_Ratio.dropna(), merged_data[merged_data['Univ_Town'] == False].Price_Ratio.dropna())
    if merged_data[merged_data['Univ_Town']].Price_Ratio.mean() < merged_data[merged_data['Univ_Town'] == False].Price_Ratio.mean():
        better = 'university town'
    else:
        better = 'non-university town'
    
    return P<0.01,P,better